**ĐỀ TÀI: ỨNG DỤNG DEEP LEARNING TRONG CÔNG NGHỆ NHẬN DIỆN GIỌNG NÓI VÀ TỐI ƯU HÓA DỊCH THUẬT NGÔN NGỮ**

Danh sách nhóm thực hiện

| STT | Họ và tên           | Lớp         | Chức danh    | Nhiệm vụ      |
|-----|---------------------|-------------|--------------|---------------|
| 1   | Lê Thị Linh Phương  | DHKL16A2HN  | Chủ nhiệm    | ND1, ND3      |
| 2   | Nguyễn Đăng Thanh   | DHKL16A2HN  | Thành viên   | ND2, ND3      |
| 3   | Nguyễn Hữu Đức      | DHKL16A2HN  | Thành viên   | ND2, ND3      |
| 4   | Nguyễn Văn Hà       | DHKL16A2HN  | Thành viên   | ND1, ND3      |


#NỘI DUNG 1: FILE ÂM THANH -> VĂN BẢN

**Trong nội dung này, trọng tâm là ứng dụng mô hình học sâu (deep learning) để chuyển đổi tín hiệu âm thanh đầu vào thành văn bản một cách tự động, chính xác và không phụ thuộc vào các dịch vụ bên ngoài.**

Đầu tiên, chúng ta cần cài đặt một số thư viện hỗ trợ như Gradio, Transformers và Torchaudio để đảm bảo quá trình nhận diện giọng nói và xử lý âm thanh diễn ra hiệu quả.

## Cài đặt các thư viện

In [ ]:
!pip install -q gradio transformers torchaudio

**Trong đó:**
- `gradio`:	Tạo giao diện web đơn giản cho ứng dụng AI, ví dụ như upload file âm thanh và hiển thị kết quả
- `transformers`:	Cung cấp mô hình deep learning (ví dụ: Wav2Vec2, MBart, MarianMT) từ thư viện Hugging Face
- `torchaudio`:	Thư viện xử lý âm thanh chuyên biệt cho PyTorch (chuyển đổi sampling rate, load WAV...)

## Import các thư viện

In [ ]:
import gradio as gr
import torch
import numpy as np
import torchaudio
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2ForCTC,
    AutoFeatureExtractor,
    Wav2Vec2ForSequenceClassification
)
import re

**Chi tiết các thư viện được import:**

- `gradio`: Tạo giao diện web đơn giản để người dùng ghi âm hoặc upload file âm thanh.
- `torch`, `torchaudio`: Xử lý tensor và dữ liệu âm thanh, chuẩn bị đầu vào cho mô hình học sâu.
- `numpy`: Thư viện toán học cơ bản, xử lý mảng số liệu.
- `transformers`: Tải và sử dụng các mô hình deep learning (như Wav2Vec2, MMS-LID) từ Hugging Face.
- `re`: Xử lý biểu thức chính quy, hỗ trợ kiểm tra nội dung văn bản đầu ra.

## Khai báo cấu hình mô hình

Trong phần này, chương trình định nghĩa danh sách các mô hình nhận diện giọng nói tương ứng với từng ngôn ngữ, bao gồm: tiếng Anh, tiếng Việt, tiếng Pháp và tiếng Hàn.

- `LANG_MODELS`: Là một từ điển ánh xạ mã ngôn ngữ (`"en"`, `"vi"`, `"fr"`, `"ko"`) với tên mô hình tương ứng được tải từ Hugging Face.

- `model_cache`, `processor_cache`: Dùng để lưu trữ các mô hình và processor sau khi tải lần đầu. Việc này giúp tránh việc tải lại mô hình nhiều lần, tiết kiệm thời gian và tăng hiệu năng xử lý.

→ Cách này hỗ trợ mở rộng dễ dàng nếu muốn thêm ngôn ngữ mới chỉ cần bổ sung vào `LANG_MODELS` mà không phải thay đổi logic chương trình.


In [ ]:
# Danh sách mô hình nhận diện giọng nói cho từng ngôn ngữ
LANG_MODELS = {
    "en": "facebook/wav2vec2-base-960h",
    "vi": "nguyenvulebinh/wav2vec2-base-vietnamese-250h",
    "fr": "jonatasgrosman/wav2vec2-large-xlsr-53-french",
    "ko": "kresnik/wav2vec2-large-xlsr-korean"
}

# Cache để lưu lại mô hình đã tải
model_cache = {}
processor_cache = {}

## Mô hình phát hiện ngôn ngữ

Để xác định ngôn ngữ nói trong tệp âm thanh, chương trình sử dụng mô hình **"facebook/mms-lid-256"** – một mô hình học sâu thuộc dự án MMS (Massively Multilingual Speech) do Meta AI phát triển.

Cấu hình mô hình:
- `LANG_ID_MODEL`: Đường dẫn đến mô hình được tải từ thư viện Hugging Face.
- `AutoFeatureExtractor`: Bộ trích xuất đặc trưng âm thanh đầu vào để phù hợp với định dạng mô hình yêu cầu.
- `Wav2Vec2ForSequenceClassification`: Mô hình phân loại chuỗi tín hiệu đầu vào thành các nhãn ngôn ngữ.
- `lid_model.eval()`: Đặt mô hình ở chế độ suy luận, không thực hiện cập nhật gradient.

Bản đồ ánh xạ ngôn ngữ (`LANG_MAP`)
- Do mô hình trả về mã ngôn ngữ theo chuẩn **ISO-639-3** (ví dụ: `"eng"`, `"vie"`...), ta ánh xạ về mã 2 ký tự đơn giản hơn (`"en"`, `"vi"`, `"fr"`, `"ko"`) để sử dụng tiếp.

→ Bước này giúp hệ thống **phân biệt được ngôn ngữ đầu vào** mà không cần người dùng chọn trước, tạo ra trải nghiệm và cảm giác tự động hơn.


In [ ]:
# Cấu hình mô hình
LANG_ID_MODEL = "facebook/mms-lid-256"
extractor = AutoFeatureExtractor.from_pretrained(LANG_ID_MODEL)
lid_model = Wav2Vec2ForSequenceClassification.from_pretrained(LANG_ID_MODEL)
lid_model.eval()
ID2LABEL = lid_model.config.id2label

# Bản đồ ngôn ngữ
LANG_MAP = {"eng": "en", "vie": "vi", "fra": "fr", "kor": "ko"}

## Các hàm xử lý phụ trợ

Phần này bao gồm các hàm hỗ trợ quá trình xử lý âm thanh đầu vào trước khi đưa vào mô hình, nhằm đảm bảo định dạng và chất lượng tín hiệu phù hợp.


In [ ]:
# Hàm đảm bảo audio có sampling rate là 16000 Hz
def ensure_16k(audio_tensor, sr):
    if sr != 16000:
        resampler = torchaudio.transforms.Resample(orig_freq=sr, new_freq=16000)
        audio_tensor = resampler(audio_tensor)
        return audio_tensor, 16000
    return audio_tensor, sr

# Hàm kiểm tra xem văn bản có phải là văn bản rác hay không
def is_gibberish(text):
    nonsense = re.findall(r"\b[A-Z]{4,}\b", text)
    return len(nonsense) > 3

**1. `ensure_16k(audio_tensor, sr)`**
- Mục đích: Đảm bảo rằng tín hiệu âm thanh có **tần số lấy mẫu (sampling rate)** đúng là `16000 Hz`, vì hầu hết các mô hình Wav2Vec2 yêu cầu chuẩn này.
- Nếu đầu vào có sampling rate khác, hàm sẽ thực hiện nội suy lại về 16000 Hz bằng `torchaudio.transforms.Resample`.

**2. `is_gibberish(text)`**
- Mục đích: Kiểm tra xem văn bản đầu ra có phải là **văn bản rác hoặc vô nghĩa** hay không.
- Cách hoạt động: Tìm các từ viết IN HOA có độ dài ≥ 4 và nếu có hơn 3 từ như vậy, thì được coi là rác.
- Ứng dụng: Hữu ích trong việc loại bỏ kết quả nhận diện sai hoặc nhiễu trong đầu ra.

## Hàm chính xử lý âm thanh

Các hàm trong phần này đảm nhiệm vai trò trung tâm trong quy trình xử lý tín hiệu âm thanh, bao gồm:

- **Phân loại ngôn ngữ** từ dữ liệu âm thanh thông qua mô hình học sâu  
- **Khởi tạo và quản lý mô hình** nhận diện giọng nói phù hợp  
- **Thực hiện chuyển đổi** giọng nói sang dạng văn bản  
- **Tích hợp toàn bộ quy trình xử lý** vào một hàm duy nhất (`process`)


In [ ]:
# Hàm phát hiện ngôn ngữ nói từ âm thanh bằng mô hình phân loại ngôn ngữ
def detect_language(audio_tensor, sr=16000):
    inputs = extractor(audio_tensor.numpy(), sampling_rate=sr, return_tensors="pt")
    with torch.no_grad():
        logits = lid_model(**inputs).logits
    pred_id = logits.argmax(dim=-1).item()
    iso3 = ID2LABEL[pred_id]
    lang = LANG_MAP.get(iso3, "en")
    print(f"[DEBUG] Detected ISO: {iso3} → lang: {lang}")
    return lang

# Hàm tải mô hình và processor tương ứng với ngôn ngữ nếu chưa có trong cache
def get_cached_model(lang):
    if lang not in model_cache:
        print(f"[INFO] Loading model for language: {lang}")
        model_name = LANG_MODELS[lang]
        processor_cache[lang] = Wav2Vec2Processor.from_pretrained(model_name)
        model_cache[lang] = Wav2Vec2ForCTC.from_pretrained(model_name).eval()
    return processor_cache[lang], model_cache[lang]

# Hàm chuyển đổi âm thanh sang văn bản bằng mô hình Wav2Vec2 đã được cache
def speech_to_text(audio_tensor, lang, sr=16000):
    processor, model = get_cached_model(lang)
    inputs = processor(audio_tensor, sampling_rate=sr, return_tensors="pt")
    with torch.no_grad():
        logits = model(**inputs).logits
    pred_ids = torch.argmax(logits, dim=-1)
    return processor.batch_decode(pred_ids)[0]

# Hàm xử lý chính: chuẩn hóa âm thanh, phát hiện ngôn ngữ, nhận diện giọng nói và trả về kết quả
def process(audio_input):
    try:
        sr, array = audio_input
        audio_tensor = torch.tensor(array, dtype=torch.float32)
        audio_tensor = audio_tensor / torch.abs(audio_tensor).max()
        audio_tensor, sr = ensure_16k(audio_tensor, sr)
        lang = detect_language(audio_tensor, sr)
        text = speech_to_text(audio_tensor, lang, sr)
        return lang.upper(), text
    except Exception as e:
        return f"Error: {e}", ""

1. `detect_language(audio_tensor, sr)`
- Sử dụng mô hình `facebook/mms-lid-256` để phát hiện ngôn ngữ từ dữ liệu âm thanh.
- Trả về mã ngôn ngữ (ví dụ: `"en"`, `"vi"`,...) dùng trong các bước tiếp theo.

2. `get_cached_model(lang)`
- Kiểm tra xem mô hình và processor cho ngôn ngữ đó đã được tải chưa.
- Nếu chưa, tải từ Hugging Face và lưu vào `cache` để dùng lại nhanh hơn sau này.

3. `speech_to_text(audio_tensor, lang, sr)`
- Dùng `Wav2Vec2Processor` để xử lý âm thanh thành tensor phù hợp.
- Cho vào mô hình `Wav2Vec2ForCTC` để suy luận và trả về văn bản tương ứng.

4. `process(audio_input)`
- Là hàm tích hợp toàn bộ pipeline:
  - Chuẩn hóa âm thanh
  - Resample về 16kHz
  - Phát hiện ngôn ngữ
  - Nhận diện giọng nói
  - Trả về kết quả cuối cùng: ngôn ngữ + văn bản

→ Đây là phần trung tâm quyết định toàn bộ quá trình hoạt động của ứng dụng nhận diện giọng nói đa ngôn ngữ.


## Tạo giao diện Gradio

Gradio được sử dụng để tạo một **giao diện web đơn giản** cho hệ thống nhận diện giọng nói. Thay vì xây dựng giao diện bằng HTML hoặc các framework phức tạp, Gradio cho phép người dùng **tương tác trực tiếp với mô hình deep learning** chỉ với vài dòng lệnh.

**Mục đích sử dụng:**

- Tạo giao diện thân thiện: Cho phép người dùng ghi âm hoặc tải file âm thanh đầu vào dễ dàng.
- Gọi hàm xử lý `process()`: Tự động thực thi toàn bộ quy trình xử lý âm thanh.
- Hiển thị kết quả ngay lập tức: Bao gồm ngôn ngữ được phát hiện và văn bản nhận diện từ giọng nói.
- Chạy trực tiếp trên Colab và chia sẻ dễ dàng: Hỗ trợ link truy cập `gradio.live` giúp demo nhanh chóng.
- Tùy chỉnh giao diện dễ dàng: Có thể thêm tiêu đề, mô tả, biểu tượng để tăng tính trực quan.

In [ ]:
# Tạo giao diện Gradio
gr.Interface(
    fn=process,
    inputs=gr.Audio(type="numpy", label="Ghi âm hoặc tải file âm thanh"),
    outputs=[
        gr.Text(label="Ngôn ngữ phát hiện"),
        gr.Text(label="Văn bản nhận được")
    ],
    title="Nhận diện giọng nói tự động nhiều ngôn ngữ (Anh - Việt - Pháp - Hàn)",
    description="Hệ thống tự động nhận diện ngôn ngữ và chuyển đổi giọng nói sang văn bản."
).launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://5cac44b5ab5f4302a8.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


[DEBUG] Detected ISO: kor → lang: ko
[INFO] Loading model for language: ko


preprocessor_config.json:   0%|          | 0.00/214 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/161 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.31k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/18.2k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.27G [00:00<?, ?B/s]

Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://5cac44b5ab5f4302a8.gradio.live


**Các thành phần chính:**

- `fn=process`: Hàm xử lý chính sẽ được gọi mỗi khi người dùng gửi file âm thanh.
- `inputs=gr.Audio(...)`: Thành phần nhập, cho phép người dùng ghi âm hoặc upload file `.wav`.
- `outputs=[...]`: Gồm hai kết quả đầu ra:
  - Văn bản hiển thị ngôn ngữ được phát hiện
  - Văn bản nhận diện từ giọng nói
- `title`, `description`: Tiêu đề và mô tả của ứng dụng.

#NỘI DUNG 2: FILE VĂN BẢN (NGÔN NGỮ NGUỒN) -> VĂN BẢN (NGÔN NGỮ ĐÍCH)

Trong nội dung này, hệ thống thực hiện chức năng **dịch tự động một đoạn văn bản đầu vào sang các ngôn ngữ còn lại**, dựa trên việc tự động phát hiện ngôn ngữ nguồn. Người dùng không cần chọn ngôn ngữ đích, hệ thống sẽ chủ động dịch sang tất cả các ngôn ngữ hỗ trợ còn lại.

**Mục tiêu:**
- Nhập văn bản bằng một ngôn ngữ bất kỳ (trong 4 ngôn ngữ: Anh, Việt, Pháp, Hàn)
- Hệ thống tự động phát hiện ngôn ngữ nguồn
- Dịch văn bản đó sang **3 ngôn ngữ còn lại**
- Hiển thị đồng thời tất cả các bản dịch

**Mô hình sử dụng:**
- Sử dụng các mô hình dịch đa ngôn ngữ như **MBart50** từ Hugging Face Transformers
- Hỗ trợ nhiều ngôn ngữ với độ chính xác cao

**Đánh giá hiệu suất bản dịch:**

Sau khi hoàn tất quá trình dịch, hệ thống tiến hành đánh giá một số khía cạnh về chất lượng và đặc điểm của các bản dịch đầu ra nhằm:

- Phân tích độ dài, độ phức tạp, độ dễ đọc
- Đo lường hiệu quả hoạt động của hệ thống dịch

→ Việc kết hợp chức năng dịch đa ngôn ngữ và đánh giá hiệu suất giúp hệ thống không chỉ tạo ra kết quả đúng, mà còn kiểm soát được **chất lượng** và **tính tự nhiên** của văn bản dịch.


##Cài đặt thư viện

Trước khi thực hiện dịch văn bản và đánh giá hiệu suất, cần cài đặt một số thư viện hỗ trợ:

- `transformers`: Cung cấp các mô hình dịch tự động như MBart hoặc MarianMT từ Hugging Face.
- `langdetect`: Dùng để phát hiện ngôn ngữ đầu vào (hữu ích trong bước dịch tự động sang 3 ngôn ngữ còn lại).
- `textstat`: Tính toán các chỉ số đánh giá văn bản như độ dễ đọc, độ dài câu, tính đa dạng từ vựng.

→ Các thư viện này giúp hệ thống thực hiện cả chức năng **dịch tự động** lẫn **đánh giá chất lượng đầu ra** một cách hiệu quả.


In [ ]:
!pip install transformers
!pip install langdetect
!pip install textstat

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993223 sha256=e55284b7b7020f5240bb1971c6c14351d389f27b724de691076e90f4e6af7a4c
  Stored in directory: /root/.cache/pip/wheels/0a/f2/b2/e5ca405801e05eb7c8ed5b3b4bcf1fcabcd6272c167640072e
Successfully built langdetect
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.3/105.3 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 939.4/939.4 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.6 MB/s eta 0:00:00


## Import các thư viện

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50Tokenizer
from langdetect import detect
import re
import time
import textstat

**Chi tiết các thư viện được import:**
- `transformers`
 - Sử dụng mô hình dịch đa ngôn ngữ `MBartForConditionalGeneration` từ Hugging Face.
 - Tokenizer `MBart50Tokenizer` dùng để mã hóa/giải mã văn bản cho 50 ngôn ngữ.
- `langdetect`
 - Dùng để phát hiện ngôn ngữ đầu vào (ví dụ: `vi`, `en`, `fr`...).
 - Giúp tự động cấu hình mã nguồn ngôn ngữ (`src_lang`) phù hợp cho tokenizer.

- `re`
 - Xử lý tiền xử lý văn bản: loại bỏ khoảng trắng dư thừa, chuẩn hóa câu, tách câu theo dấu chấm/hỏi.

- `time`
 - Đo thời gian dịch của từng văn bản (tính bằng giây).
 - Hỗ trợ đánh giá hiệu suất mô hình dịch.

- `textstat`
 - Đánh giá độ dễ đọc của văn bản dịch.
 - Dùng để so sánh chất lượng ngôn ngữ đầu ra giữa các bản dịch.


## Khởi tạo lớp dịch ngôn ngữ


**Mục tiêu:**

- `TranslationModel` là một lớp Python được xây dựng để **tự động phát hiện ngôn ngữ của văn bản đầu vào** và **dịch văn bản đó sang các ngôn ngữ khác** (Anh, Việt, Pháp, Hàn) bằng mô hình deep learning MBart từ thư viện Hugging Face.

- Mô hình dịch này hoạt động theo hướng **đầu vào văn bản → tiền xử lý → mô hình sinh đầu ra bằng ngôn ngữ đích**.

In [ ]:
# Phần 1: Khởi tạo model và tokenizer
class TranslationModel:
    def __init__(self):
        # Khởi tạo model MBart và tokenizer tương ứng với 50 ngôn ngữ
        self.model_name = "facebook/mbart-large-50-many-to-many-mmt"
        self.tokenizer = MBart50Tokenizer.from_pretrained(self.model_name)
        self.model = MBartForConditionalGeneration.from_pretrained(self.model_name)
        self.lang_map = {
            "vi": "vi_VN",
            "en": "en_XX",
            "fr": "fr_XX",
            "ko": "ko_KR"
        }


    # Hàm phát hiện ngôn ngữ của văn bản đầu vào
    # Trả về mã ngôn ngữ chuẩn hóa theo định dạng MBart
    def detect_language(self, text):
        detected = detect_langs(text)
        if not detected:
            return "vi_VN"   # Mặc định là tiếng Việt nếu không nhận diện được
        lang_code = detected[0].lang
        return self.lang_map.get(lang_code, "vi_VN")


    # Hàm tiền xử lý văn bản: chuẩn hóa khoảng trắng và viết hoa đầu câu
    # Mục đích: làm sạch đầu vào giúp model dịch chính xác hơn
    def preprocess_text(self, text):
        text = re.sub(r'\s+', ' ', text).strip()   # Xóa khoảng trắng dư
        sentences = re.split(r'(?<=[.!?])\s+', text)   # Tách câu theo dấu câu
        processed_sentences = [s.capitalize() for s in sentences if s]   # Viết hoa mỗi câu
        return ' '.join(processed_sentences)


    # Hàm dịch văn bản từ ngôn ngữ gốc sang ngôn ngữ đích
    # Gồm các bước: phát hiện ngôn ngữ, tiền xử lý, token hóa, dịch, giải mã
    def translate_text(self, text, target_language):
        src_lang = self.detect_language(text)
        self.tokenizer.src_lang = src_lang

        text = self.preprocess_text(text)

        start_time = time.time()
        encoded_text = self.tokenizer(text, return_tensors="pt", max_length=512, truncation=True)

        # Tạo chuỗi văn bản dịch bằng cách sử dụng beam search với ngôn ngữ đích
        generated_tokens = self.model.generate(
            **encoded_text,
            forced_bos_token_id=self.tokenizer.lang_code_to_id[target_language],
            max_new_tokens=100,
            no_repeat_ngram_size=3,
            early_stopping=True,
            num_beams=4
        )
        end_time = time.time()

        # Giải mã token đầu ra thành văn bản
        translated_text = self.tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        translation_time = end_time - start_time

        return translated_text, src_lang, translation_time

**Các bước chính trong class**

1. `__init__()`: **Khởi tạo mô hình và cấu hình cần thiết**
- Tải mô hình MBart đã được huấn luyện cho 50 ngôn ngữ từ Hugging Face.
- Khởi tạo tokenizer để mã hóa văn bản thành token số.
- Tạo bản đồ ánh xạ ngôn ngữ (`lang_map`) để chuyển đổi các mã ngôn ngữ thông thường như `en`, `vi` sang định dạng mà MBart yêu cầu như `en_XX`, `vi_VN`.

→ Vì MBart không hiểu mã `"en"` hay `"vi"`, nó chỉ hoạt động đúng khi được chỉ định ngôn ngữ theo chuẩn riêng của nó.

2. `detect_language()`: **Tự động phát hiện ngôn ngữ đầu vào**

- Sử dụng `langdetect` để đoán ngôn ngữ của đoạn văn bản.
- Trả về mã ngôn ngữ tương ứng với định dạng MBart để đặt `tokenizer.src_lang`.

→  Nếu không xác định đúng ngôn ngữ gốc thì mô hình có thể dịch sai hoàn toàn.  Việc này giúp hệ thống **tự động hóa** thay vì yêu cầu người dùng chọn tay.


3. `preprocess_text()`: **Làm sạch văn bản đầu vào**

- Xóa khoảng trắng dư thừa, chuẩn hóa định dạng câu.
- Tách văn bản thành các câu ngắn, viết hoa chữ cái đầu mỗi câu.

→  Mô hình dịch hoạt động tốt hơn khi văn bản có cấu trúc rõ ràng, tránh các lỗi định dạng gây khó hiểu cho model.


4. `translate_text()`: **Dịch văn bản từ ngôn ngữ gốc sang ngôn ngữ đích**

Quy trình gồm 4 bước:
1. **Phát hiện ngôn ngữ gốc** (`detect_language`)
2. **Tiền xử lý văn bản** (`preprocess_text`)
3. **Mã hóa đầu vào** → Token hóa → Dịch bằng MBart (`model.generate`)
4. **Giải mã đầu ra** → Chuyển token thành văn bản (`tokenizer.decode`)
5. **Tính thời gian xử lý**

→ Đây là quy trình dịch hoàn chỉnh sử dụng mô hình deep learning dạng seq2seq (sequence-to-sequence). Vì phải chỉ định đúng ngôn ngữ đích thì mô hình mới sinh ra câu đúng ngữ nghĩa.


## Chương trình chính, xử lý dịch ngôn ngữ và đánh giá hiệu suất

Hàm `main()` thực hiện toàn bộ quy trình tương tác với người dùng:
- Nhận văn bản đầu vào từ người dùng
- Tự động phát hiện ngôn ngữ đầu vào bằng `detect_language()`
- Dịch văn bản sang các ngôn ngữ còn lại trong danh sách `target_languages`
- Hiển thị văn bản gốc, văn bản dịch và thời gian thực hiện từng bản dịch
- Đánh giá hiệu suất của từng bản dịch

→ Mục tiêu: tạo nên một pipeline dịch tự động đầu-cuối, hỗ trợ hiệu quả cho người dùng.

In [ ]:
# Phần 2: Nhập văn bản và dịch sang nhiều ngôn ngữ
def main():
    original_text = input("Nhập văn bản cần dịch: ").strip()
    if not original_text:
        print("Không có nội dung để dịch.")
        return

    model = TranslationModel()
    detected_lang = model.detect_language(original_text)
    print(f"\nNgôn ngữ phát hiện (src): {detected_lang}")

    target_languages = {
        "Tiếng Việt": "vi_VN",
        "Tiếng Anh": "en_XX",
        "Tiếng Pháp": "fr_XX",
        "Tiếng Hàn": "ko_KR"
    }

    translations = {}
    for lang, code in target_languages.items():
        if code == detected_lang:
            continue  # Bỏ qua nếu ngôn ngữ đích trùng với ngôn ngữ gốc
        translated_text, _, translation_time = model.translate_text(original_text, code)
        translations[lang] = (translated_text, translation_time)

    print("\nNội dung gốc:")
    print(original_text)
    for language, (translated_text, translation_time) in translations.items():
        print(f"\nVăn bản dịch sang {language}:")
        print(translated_text)
        print(f"Thời gian dịch: {translation_time:.4f} giây")

# Phần 3: Đánh giá hiệu suất

# Gọi hàm đánh giá hiệu suất sau khi dịch
    evaluate_translations(translations, original_text)
def evaluate_translations(translations, original_text):
    print("\n--- Đánh giá hiệu suất ---")
    for language, (translated_text, translation_time) in translations.items():
        print(f"\n{language}:")

        # Tính độ dài văn bản gốc và văn bản dịch
        original_word_count = len(original_text.split())
        translated_word_count = len(translated_text.split())

        # Tính điểm dễ đọc
        readability_score = textstat.flesch_reading_ease(translated_text)

        # Tính đa dạng từ vựng (số từ duy nhất / tổng số từ)
        unique_words = set(translated_text.split())
        lexical_diversity = len(unique_words) / translated_word_count if translated_word_count else 0

        # Tính độ dài trung bình câu
        sentences = [s for s in translated_text.split('.') if s.strip()]
        sentence_lengths = [len(s.split()) for s in sentences]
        avg_sentence_length = sum(sentence_lengths) / max(1, len(sentences))

        # In kết quả
        print(f"- Độ dài văn bản gốc: {original_word_count} từ")
        print(f"- Độ dài văn bản dịch: {translated_word_count} từ")
        print(f"- Độ dễ đọc: {readability_score:.2f}")
        print(f"- Tính đa dạng từ vựng: {lexical_diversity:.2f}")
        print(f"- Độ dài trung bình câu: {avg_sentence_length:.2f} từ/câu")
        print(f"- Thời gian dịch: {translation_time:.4f} giây")
if __name__ == "__main__":
    main()

Nhập văn bản cần dịch: Sáng nay thời tiết rất đẹp. Tôi quyết định đi dạo quanh công viên để thư giãn và hít thở không khí trong lành.

Ngôn ngữ phát hiện (src): vi_VN

Nội dung gốc:
Sáng nay thời tiết rất đẹp. Tôi quyết định đi dạo quanh công viên để thư giãn và hít thở không khí trong lành.

Văn bản dịch sang Tiếng Anh:
This morning it was nice. I decided to go for a walk around the park to relax and to breathe in the fresh air.
Thời gian dịch: 13.9844 giây

Văn bản dịch sang Tiếng Pháp:
Il y a beau temps ce matin, et j'ai décidé de marcher dans le parc pour me détendre et respirer.
Thời gian dịch: 15.4097 giây

Văn bản dịch sang Tiếng Hàn:
오늘 날씨가 좋네요. 저는 산책을 하기로 결심했습니다. 쾌적한 공기를 들을 수 있도록 말이죠.
Thời gian dịch: 14.4913 giây

--- Đánh giá hiệu suất ---

Tiếng Anh:
- Độ dài văn bản gốc: 24 từ
- Độ dài văn bản dịch: 24 từ
- Độ dễ đọc: 93.14
- Tính đa dạng từ vựng: 0.88
- Độ dài trung bình câu: 12.00 từ/câu
- Thời gian dịch: 13.9844 giây

Tiếng Pháp:
- Độ dài văn bản gốc: 24 từ
- Độ dài văn 

**Đánh giá hiệu suất bản dịch**

Sau khi hoàn tất quá trình dịch, chương trình thực hiện đánh giá hiệu suất đầu ra của từng bản dịch thông qua hàm `evaluate_translations()` với các chỉ số quan trọng sau:

1. **Độ dài văn bản gốc và bản dịch**:  
- So sánh số từ giúp đánh giá mức độ súc tích hay dài dòng của bản dịch so với nội dung gốc.

2. **Độ dễ đọc:**
- Sử dụng chỉ số `readability_score` để đo mức độ dễ hiểu của bản dịch.  
- Giá trị càng cao → văn bản càng dễ đọc.  
- Được tính theo công thức:

$$
\text{Score} = 206.835 - 1.015 \times \left( \frac{\text{Tổng số từ}}{\text{Tổng số câu}} \right) - 84.6 \times \left( \frac{\text{Tổng số âm tiết}}{\text{Tổng số từ}} \right)
$$

**Trong đó:**
- Tổng số từ: tổng số lượng từ trong văn bản
- Tổng số câu: đếm theo dấu câu `. ! ?`
- Tổng số âm tiết: đếm số âm tiết trong từng từ (ước lượng theo thư viện `textstat`)

3. **Tính đa dạng từ vựng**:  
Tính theo công thức:
$$
\text{Lexical Diversity} = \frac{\text{Số từ duy nhất}}{\text{Tổng số từ}}
$$

**Trong đó:**
- Số từ duy nhất: là số lượng từ không trùng lặp trong văn bản
- Tổng số từ: toàn bộ từ trong văn bản, bao gồm cả từ lặp

4. **Độ dài trung bình câu (Average Sentence Length)**:  
- Trung bình số từ trên mỗi câu.
- Giá trị cao có thể chỉ ra văn phong học thuật hoặc văn bản dài dòng; giá trị thấp thường là ngôn ngữ đơn giản, trực tiếp.
- Được tính theo công thức:

$$
\text{Average Sentence Length} = \frac{\text{Tổng số từ}}{\text{Tổng số câu}}
$$

**Trong đó:**
- Tổng số từ: toàn bộ từ trong văn bản
- Tổng số câu: đếm bằng cách phân chia theo các dấu câu kết thúc như `. ! ?`

5. **Thời gian dịch (Translation Time)**:  
  Là thời gian tính bằng giây để mô hình sinh ra bản dịch. Giá trị này giúp đánh giá hiệu suất thực thi của hệ thống.

→ Các chỉ số trên không chỉ phản ánh chất lượng ngôn ngữ đầu ra, mà còn giúp người dùng có cái nhìn định lượng để so sánh các bản dịch giữa các ngôn ngữ khác nhau hoặc giữa các mô hình dịch khác nhau.

#NỘI DUNG 3: TÍCH HỢP NHẬN DIỆN GIỌNG NÓI - DỊCH THUẬT NGÔN NGỮ

Ở nội dung 3, nhóm ban đầu dự định sử dụng các mô hình deep learning như Wav2Vec2 (nhận diện giọng nói) và MBart (dịch ngôn ngữ) để xây dựng một pipeline xử lý âm thanh – dịch – hiển thị hoàn toàn tự động.

Tuy nhiên, trong quá trình thực hiện, nhóm gặp phải hạn chế về hiệu năng và tài nguyên khi triển khai mô hình deep learning trên môi trường VS Code:

- Các mô hình như Wav2Vec2 yêu cầu dung lượng RAM lớn, khả năng xử lý song song (GPU) và nhiều thư viện chuyên dụng về xử lý tín hiệu âm thanh.

- Quá trình tải mô hình, khởi tạo tokenizer, và convert định dạng âm thanh tốn nhiều thời gian và không đảm bảo ổn định trong điều kiện thiết bị hiện có.


Vì vậy, nhóm chuyển sang sử dụng các thư viện nhẹ hơn như `speech_recognition`(dựa trên Google Web Speech API) và googletrans để đảm bảo khả năng chạy mượt, dễ thử nghiệm và dễ tích hợp vào giao diện trong môi trường VS Code.
Trong tương lai, nếu được triển khai trên môi trường mạnh hơn (Google Colab Pro, server GPU hoặc nền tảng cloud), nhóm hoàn toàn có thể tái cấu trúc lại hệ thống để tích hợp hoàn toàn các mô hình deep learning như mong muốn ban đầu.

Trong tương lai, nếu được triển khai trên môi trường mạnh hơn (Google Colab Pro, server GPU hoặc nền tảng cloud), nhóm có thể tái cấu trúc lại hệ thống để tích hợp hoàn toàn các mô hình deep learning như mong muốn ban đầu.

Dù không sử dụng deep learning, chương trình mà nhóm phát triển vẫn có cải tiến rõ rệt so với các công cụ dịch truyền thống nhờ vào:
- **Khả năng kết hợp nhận diện giọng nói + dịch ngôn ngữ + phản hồi văn bản** trong cùng một pipeline
- **Tối ưu hóa luồng xử lý**, tự động hóa từ đầu vào giọng nói đến đầu ra là bản dịch, thay vì yêu cầu thao tác thủ công từng bước
- Có thể tích hợp vào các ứng dụng thực tế như hỗ trợ giao tiếp song ngữ, dịch hội thoại.


## Cài đặt thư viện

In [ ]:
!pip install googletrans==4.0.0-rc1 gtts
!pip install speechrecognition
!pip install pygame
!pip install tkinker
!pip install pyaudio

## Import các thư viện

In [ ]:
import tkinter as tk
from tkinter import ttk
from tkinter import messagebox
import speech_recognition as sr
from googletrans import Translator
from gtts import gTTS
import pygame
import os

**Các thư viện được sử dụng trong chương trình bao gồm:**

- `tkinter`, `ttk`, `messagebox`: dùng để xây dựng **giao diện đồ họa (GUI)** tương tác với người dùng
- `speech_recognition`: thư viện **nhận diện giọng nói** từ microphone đầu vào
- `googletrans`: sử dụng API không chính thức của Google Translate để **dịch ngôn ngữ**
- `gtts`: dùng để **chuyển văn bản thành giọng nói**
- `pygame`: hỗ trợ **phát âm thanh** từ file `.mp3` sau khi chuyển văn bản thành tiếng
- `os`: xử lý các thao tác hệ thống như lưu, xóa file âm thanh tạm thời

→ Việc kết hợp các thư viện trên giúp chương trình có thể thực hiện đầy đủ quy trình:  
> **nghe giọng nói ➜ chuyển thành văn bản ➜ dịch ngôn ngữ ➜ phát âm bản dịch**.


## Ứng dụng GUI: Nhận diện giọng nói và dịch ngôn ngữ đa ngữ


Chương trình này sử dụng thư viện `tkinter` để xây dựng giao diện tương tác, kết hợp với:
- `speech_recognition` để nhận diện giọng nói từ micro
- `googletrans` để dịch văn bản sang nhiều ngôn ngữ
- `gtts` + `pygame` để đọc lại nội dung đã nhận hoặc dịch

Người dùng có thể:
- Chọn ngôn ngữ cần nói
- Bấm ghi âm → văn bản được hiển thị
- Chọn ngôn ngữ cần dịch (chọn được nhiều ngôn ngữ) → bản dịch hiện ra và được phát âm lại

**So sánh với công cụ dịch thuật truyền thống (ví dụ: Google Dịch):**

| Tiêu chí                            | Công cụ truyền thống      | Ứng dụng nhóm phát triển |
|------------------------------------|----------------------------|---------------------------|
| Nhận diện giọng nói                | ✅ Có | ✅ Có |
| Dịch đa ngôn ngữ cùng lúc          | ❌ Dịch từng ngôn ngữ thủ công | ✅ Dịch nhiều ngôn ngữ chỉ với 1 thao tác |
| Phản hồi âm thanh bản dịch         | ❌ Giới hạn  | ✅ Tự động phát âm sau dịch |
| Tùy biến giao diện người dùng (UI) | ❌ Không                   | ✅ Có thể chỉnh sửa, tích hợp thêm chức năng |
| Tính chủ động (offline, gọn nhẹ)   | ❌ Phụ thuộc trình duyệt   | ✅ Chạy độc lập trên máy |

In [ ]:
# Hàm chuyển giọng nói thành văn bản
def recognize_speech(language):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        messagebox.showinfo("Hướng dẫn", "Vui lòng nói sau khi nghe tiếng bíp", parent=app)
        try:
            recognizer.adjust_for_ambient_noise(source)
            audio = recognizer.listen(source)
            text = recognizer.recognize_google(audio, language=language)
            return text
        except sr.UnknownValueError:
            return "Không thể nhận diện giọng nói."
        except sr.RequestError:
            return "Lỗi kết nối đến dịch vụ nhận diện giọng nói."

# Hàm đọc văn bản với ngôn ngữ cụ thể (sử dụng gTTS và pygame)
def speak_text(text, language):
    try:
        tts = gTTS(text=text, lang=language, slow=False)
        temp_audio_file = "temp_audio.mp3"
        tts.save(temp_audio_file)

        pygame.mixer.init()
        pygame.mixer.music.load(temp_audio_file)
        pygame.mixer.music.play()

        while pygame.mixer.music.get_busy():
            continue

        pygame.mixer.music.unload()
        os.remove(temp_audio_file)
    except Exception as e:
        messagebox.showerror("Lỗi", f"Không thể đọc văn bản: {e}")

# Hàm xử lý giọng nói thành văn bản
def handle_speech_to_text():
    language_mapping = {
        "Tiếng Anh": "en",
        "Tiếng Pháp": "fr",
        "Tiếng Hàn": "ko",
        "Tiếng Việt": "vi",
    }

    selected_language = language_var.get()
    if not selected_language:
        messagebox.showerror("Lỗi", "Vui lòng chọn ngôn ngữ trước khi nói.")
        return

    recognized_text = recognize_speech(language_mapping[selected_language])
    text_output.delete(1.0, tk.END)
    text_output.insert(tk.END, recognized_text)
    speak_text(recognized_text, language_mapping[selected_language])  # Đọc văn bản theo ngôn ngữ

# Hàm dịch văn bản
def translate_text(text, dest_language):
    translator = Translator()
    try:
        translated = translator.translate(text, dest=dest_language)
        return translated.text
    except Exception as e:
        return f"Lỗi khi dịch: {e}"

# Hàm xử lý dịch văn bản
def handle_translate():
    language_mapping = {
        "Tiếng Anh": "en",
        "Tiếng Pháp": "fr",
        "Tiếng Hàn": "ko",
        "Tiếng Việt": "vi",
    }

    selected_languages = [lang for lang, var in translation_vars.items() if var.get() == 1]
    if not selected_languages:
        messagebox.showerror("Lỗi", "Vui lòng chọn ít nhất một ngôn ngữ để dịch.")
        return

    input_text = text_output.get(1.0, tk.END).strip()
    if not input_text:
        messagebox.showerror("Lỗi", "Vui lòng nhập hoặc chuyển giọng nói thành văn bản trước khi dịch.")
        return

    translated_output.delete(1.0, tk.END)
    for lang in selected_languages:
        dest_language = language_mapping[lang]
        translated_text = translate_text(input_text, dest_language)
        translated_output.insert(tk.END, f"{lang}: {translated_text}\n")
        speak_text(translated_text, dest_language)  # Đọc văn bản đã dịch

# Giao diện chính
app = tk.Tk()
app.title("Ứng dụng chuyển giọng nói sang văn bản và dịch ngôn ngữ")

# Phần chọn ngôn ngữ giọng nói
language_var = tk.StringVar()
language_label = ttk.Label(app, text="Chọn ngôn ngữ để nói:")
language_label.pack()

language_options = ["Tiếng Anh", "Tiếng Pháp", "Tiếng Hàn", "Tiếng Việt"]
for option in language_options:
    ttk.Radiobutton(app, text=option, variable=language_var, value=option).pack(anchor=tk.W)

# Nút chuyển giọng nói thành văn bản
speech_to_text_button = ttk.Button(app, text="Chuyển giọng nói thành văn bản", command=handle_speech_to_text)
speech_to_text_button.pack(pady=10)

# Ô hiển thị văn bản
text_output = tk.Text(app, height=10, width=50)
text_output.pack(pady=10)

# Phần chọn ngôn ngữ dịch
translation_vars = {lang: tk.IntVar() for lang in language_options}
translation_label = ttk.Label(app, text="Chọn ngôn ngữ để dịch sang:")
translation_label.pack()

for option in language_options:
    ttk.Checkbutton(app, text=option, variable=translation_vars[option]).pack(anchor=tk.W)

# Nút dịch văn bản
translate_button = ttk.Button(app, text="Dịch văn bản", command=handle_translate)
translate_button.pack(pady=10)

# Ô hiển thị văn bản đã dịch
translated_output = tk.Text(app, height=10, width=50)
translated_output.pack(pady=10)

app.mainloop()

**Các thành phần và chức năng chính:**

1. `recognize_speech(language)`
- Nhận diện giọng nói từ micro sử dụng thư viện `speech_recognition`
- Tự động điều chỉnh theo tiếng ồn môi trường (`adjust_for_ambient_noise`)
- Dùng `recognize_google()` để chuyển audio thành văn bản theo ngôn ngữ đã chọn
- Báo lỗi nếu không nghe thấy hoặc không kết nối được

2. `speak_text(text, language)`
- Dùng `gTTS` để chuyển văn bản sang file âm thanh `.mp3` theo ngôn ngữ chỉ định
- Phát âm thanh đó bằng `pygame.mixer`
- Tự động xóa file sau khi phát để tiết kiệm tài nguyên

3. `handle_speech_to_text()`
- Hàm trung gian xử lý khi người dùng bấm nút "Chuyển giọng nói thành văn bản"
- Gọi `recognize_speech()` và đưa kết quả vào `text_output`
- Tự động phát lại câu vừa nhận dạng bằng `speak_text()`

4. `translate_text(text, dest_language)`
- Sử dụng `googletrans` để dịch văn bản đầu vào sang ngôn ngữ đích
- Trả về văn bản đã dịch (hoặc thông báo lỗi nếu có)

5. `handle_translate()`
- Hàm trung gian xử lý khi người dùng chọn ngôn ngữ dịch và bấm nút "Dịch"
- Lấy nội dung văn bản gốc từ `text_output`
- Dịch lần lượt sang từng ngôn ngữ được chọn
- Hiển thị các bản dịch vào `translated_output`
- Đồng thời gọi `speak_text()` để phát lại âm thanh bản dịch


## Kết luận

Ứng dụng được xây dựng trong nội dung này là minh chứng cho khả năng **tích hợp hiệu quả nhiều công cụ xử lý ngôn ngữ truyền thống** để tạo ra một pipeline đơn giản nhưng **thực tiễn và tiện dụng**.

Mặc dù **không sử dụng các mô hình deep learning phức tạp**, hệ thống vẫn thể hiện được:
- Tính tự động hóa cao: từ thu âm → nhận dạng → dịch đa ngôn ngữ → phát lại
- Giao diện thân thiện, dễ thao tác, phù hợp cho người dùng phổ thông
- Khả năng mở rộng dễ dàng: có thể nâng cấp để xử lý nâng cao hơn như lưu lịch sử, hỗ trợ nhiều ngôn ngữ hơn, hoặc chuyển sang mô hình deep learning khi có điều kiện hạ tầng

> Tổng thể, chương trình đã hoàn thành **mục tiêu nghiên cứu ban đầu**: tạo ra một công cụ chuyển giọng nói thành văn bản và dịch ngôn ngữ nhanh chóng, tiện lợi, đồng thời là nền tảng tốt để phát triển các ứng dụng giao tiếp song ngữ thông minh trong tương lai.


## Những khó khăn khi thực hiện bài nghiên cứu này và cách chúng tôi đã khắc phục

**1. Hạn chế về tài nguyên phần cứng (RAM, GPU):**
- Các mô hình deep learning như Wav2Vec2, MBart yêu cầu bộ nhớ lớn và khả năng xử lý mạnh, gây khó khăn khi chạy trên máy cá nhân.
- **Khắc phục:** Chuyển sang sử dụng các thư viện nhẹ hơn như `speech_recognition`, `googletrans` cho phần tích hợp, đồng thời tối ưu hóa việc cache mô hình và giảm kích thước dữ liệu đầu vào.

**2. Thời gian tải và khởi tạo mô hình lâu:**
- Việc tải mô hình từ Hugging Face và khởi tạo tokenizer mất nhiều thời gian, ảnh hưởng đến trải nghiệm người dùng.
- **Khắc phục:** Sử dụng cache để lưu trữ mô hình và processor sau lần tải đầu tiên, tránh tải lại nhiều lần.

**3. Xử lý định dạng và chất lượng âm thanh đầu vào:**
- Các mô hình yêu cầu âm thanh đầu vào phải đúng định dạng (sampling rate 16kHz, mono).
- **Khắc phục:** Xây dựng hàm tiền xử lý tự động chuẩn hóa âm thanh về đúng định dạng trước khi đưa vào mô hình.

**4. Độ chính xác nhận diện và dịch phụ thuộc vào chất lượng dữ liệu:**
- Nếu âm thanh nhiễu hoặc văn bản đầu vào không chuẩn hóa, kết quả nhận diện/dịch có thể sai lệch.
- **Khắc phục:** Thêm các bước kiểm tra, làm sạch dữ liệu đầu vào, loại bỏ văn bản rác và chuẩn hóa câu chữ.

**5. Tích hợp nhiều thư viện và mô hình khác nhau:**
- Việc kết hợp các thư viện truyền thống với mô hình deep learning có thể gây xung đột hoặc lỗi phụ thuộc.
- **Khắc phục:** kiểm soát phiên bản thư viện, kiểm thử từng thành phần độc lập trước khi tích hợp.

**6. Giao diện người dùng và trải nghiệm sử dụng:**
- Việc xây dựng giao diện trực quan, dễ sử dụng cho người dùng không chuyên là một thách thức.
- **Khắc phục:** Sử dụng Gradio và Tkinter để tạo giao diện web và desktop đơn giản, thân thiện, dễ thao tác.


#### Dịch thuật nâng cao trong bài này là việc sử dụng các mô hình deep learning hiện đại (như MBart, MarianMT)

    "Dịch thuật nâng cao trong bài là sử dụng mô hình deep learning để tự động phát hiện ngôn ngữ nguồn, "
    "dịch sang nhiều ngôn ngữ đích cùng lúc và đánh giá chất lượng bản dịch bằng các chỉ số định lượng "
    "(độ dễ đọc, đa dạng từ vựng, độ dài câu...). Điều này giúp tăng tính tự động hóa, hiệu quả và kiểm soát chất lượng bản dịch."
